In [1]:
from pyrosm import OSM, get_data
import osmnx as ox
import networkx as nx
import geopandas as gpd
import requests

In [37]:
scores_by_highway = {
    "cycleway":1,
    "pedestrian":0.9,
    "residential":0.8,
    "living_street":0.8,
    "tertiary":0.7,
    "tertiary_link":0.7,
    "bus_stop":0.7,
    "track":0.6,
    "services":0.6,
    "service":0.6,
    "path":0.6,
    "secondary":0.6,
    "secondary_link":0.6,
    "unclassified":0.5,
    "road":0.5,
    "primary":0.4,
    "primary_link":0.4
}

In [2]:
data = OSM( get_data("Bordeaux", directory="data"))

In [3]:
cycling_nodes, cycling_edges = data.get_network(network_type="cycling",nodes=True)

In [4]:
cycling_nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 436062 entries, 0 to 436061
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   lat        436062 non-null  float64 
 1   version    436062 non-null  int64   
 2   changeset  436062 non-null  int64   
 3   lon        436062 non-null  float64 
 4   visible    436062 non-null  bool    
 5   timestamp  436062 non-null  int64   
 6   tags       45107 non-null   object  
 7   id         436062 non-null  int64   
 8   geometry   436062 non-null  geometry
dtypes: bool(1), float64(2), geometry(1), int64(4), object(1)
memory usage: 27.0+ MB


In [5]:
cycling_nodes = cycling_nodes[["id","geometry","lat","lon"]]

In [6]:
cycling_edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 470644 entries, 0 to 470643
Data columns (total 41 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   access          25757 non-null   object  
 1   bicycle         27107 non-null   object  
 2   bicycle_road    85 non-null      object  
 3   bridge          1346 non-null    object  
 4   busway          690 non-null     object  
 5   cycleway        10003 non-null   object  
 6   est_width       17 non-null      object  
 7   foot            17643 non-null   object  
 8   footway         611 non-null     object  
 9   highway         470644 non-null  object  
 10  int_ref         250 non-null     object  
 11  junction        20639 non-null   object  
 12  lanes           37761 non-null   object  
 13  lit             37103 non-null   object  
 14  maxspeed        72936 non-null   object  
 15  motorcar        52 non-null      object  
 16  motorroad       218 non-null  

In [35]:
cycling_edges = cycling_edges[["id","u","v","geometry","highway","cycleway","surface","bicycle","length"]]
cycling_edges = cycling_edges[~cycling_edges["highway"].isin(["construction","trunk_link","trunk","footway","steps","bridleway"])]

In [8]:
graph = data.to_graph(cycling_nodes, cycling_edges, graph_type="networkx")

c:\Users\leo\OneDrive - Ynov\Cours\B3\cycliste-bordeaux-routing\venv\Lib\site-packages\pyrosm\graphs.py:155: UserWarning: Column 'oneway' missing in the edges GeoDataFrame. Assuming all edges to be bidirectional (travel allowed to both directions).
  nodes, edges = get_directed_edges(


In [34]:
def get_localisations(address):
    response = requests.get(f"https://api-adresse.data.gouv.fr/search/?q={address.replace(' ','+')}&lat=44.841225&lon=-0.580036&limit=5").json().get('features')
    response = response[0].get('geometry').get('coordinates')
    return response

In [10]:
source_address = "Quinconces, Bordeaux, France"
target_address = "Place de la Victoire, Bordeaux, France"

In [11]:
source_coordinates = get_localisations(source_address)
target_coordinates = get_localisations(target_address)

In [12]:
source_node_id = ox.nearest_nodes(graph, source_coordinates[0], source_coordinates[1])
target_node_id = ox.nearest_nodes(graph, target_coordinates[0], target_coordinates[1])

In [13]:
print(source_node_id, target_node_id)

8656225270 5898883833


In [15]:
source_node = cycling_nodes[cycling_nodes['id'] == source_node_id]
target_node = cycling_nodes[cycling_nodes['id'] == target_node_id]

In [24]:
print(source_node,target_node)

                id                   geometry        lat       lon
165143  8656225270  POINT (-0.57599 44.84542)  44.845421 -0.575985                 id                   geometry        lat       lon
315475  5898883833  POINT (-0.57271 44.83100)  44.831001 -0.572714


In [21]:
route = nx.shortest_path(graph,source_node_id,target_node_id,weight='length')

In [25]:
print(route)

[8656225270, 1914126685, 7020549386, 3280362605, 1913940753, 1914126670, 229913014, 1914126662, 1914126660, 1914126654, 1914126652, 1914126649, 1914126646, 1914126645, 8656225275, 1914126644, 1914383431, 1914383430, 1914383427, 1914383426, 1914162769, 1914383425, 1914383423, 1918498208, 1323383429, 8658614608, 4928760531, 1914383420, 1918498110, 1918498100, 1920123023, 6504150614, 7096047735, 267171552, 628597180, 1918787920, 2072627151, 2072627147, 35258390, 9503023348, 563241910, 35258655, 267243421, 35258637, 281938555, 35258503, 35258537, 35258533, 35258473, 35258606, 2046155170, 255273229, 7254773129, 2046155157, 563232961, 35258439, 2074395134, 2072636015, 2072636013, 2074395127, 250575095, 570217010, 35258427, 255272978, 253138390, 563840162, 2094985214, 249792135, 5905985708, 563840166, 2485558934, 252391533, 255544629, 255544630, 2485569772, 2615980435, 1960653605, 249792134, 8768300605, 5898665084, 9520206400, 4935524555, 4935524556, 4068957236, 5898883833]


In [27]:
import folium
m = folium.Map(location=[44.841225, -0.580036], zoom_start=13)

In [28]:
for i in range(len(route)-1) :
    edge = []
    graph_edge = graph.edges[route[i],route[i+1],0]
    for x,y in graph_edge["geometry"].coords:
        edge.append([y,x])
    tooltip_data = {"cycleway" : graph_edge["cycleway"], "surface" : graph_edge["surface"], "bicycle" : graph_edge["bicycle"], "length" : graph_edge["length"]}
    tooltip = folium.Tooltip(tooltip_data)
    folium.PolyLine(locations=edge, color='blue',tooltip=tooltip).add_to(m)


In [29]:
folium.Marker(location=[source_node.geometry.values[0].y, source_node.geometry.values[0].x], popup="source").add_to(m)
folium.Marker(location=[target_node.geometry.values[0].y, target_node.geometry.values[0].x], popup="target").add_to(m)

In [30]:
m

In [31]:
cycling_edges.head()

,id,u,v,geometry,highway,cycleway,surface,bicycle,length,oneway
0,4645906,5693915680,1710078374,"LINESTRING (-0.70774 44.89340, -0.70745 44.89318)",residential,None,asphalt,None,33.622,None
1,4645906,1710078374,29556603,"LINESTRING (-0.70745 44.89318, -0.70734 44.89312)",residential,None,asphalt,None,10.337,None
2,4645906,29556603,1828300760,"LINESTRING (-0.70734 44.89312, -0.70552 44.89185)",residential,None,asphalt,None,201.913,None
3,4645906,1828300760,29556553,"LINESTRING (-0.70552 44.89185, -0.70472 44.89130)",residential,None,asphalt,None,87.352,None
4,4645906,29556553,29556534,"LINESTRING (-0.70472 44.89130, -0.70359 44.89051)",residential,None,asphalt,None,125.730,None


In [33]:
cycling_edges["highway"].unique()

array(['residential', 'tertiary', 'secondary', 'primary', 'unclassified',
       'trunk_link', 'trunk', 'secondary_link', 'cycleway',
       'primary_link', 'service', 'living_street', 'pedestrian',
       'tertiary_link', 'footway', 'path', 'track', 'steps', 'bridleway',
       'road', 'bus_stop', 'services', 'construction'], dtype=object)